In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import cv2
print("whether to support cuda")
print(torch.cuda.is_available()) # 输出是否支持cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 自动识别使用cuda还是cpu

True


| 参数 | 说明   | 
| ----- | --------- | 
| root | 代表训练集/测试集路径 |
| train  | True代表训练集, False代表测试集 |
| transform | 将图片image转换成Tensor |
| download | 是否下载，第一次使用需要 |

In [ ]:
train_dataset = datasets.MNIST(root='./data/',train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.MNIST(root='./data/',train=False,transform=transforms.ToTensor(),download=True)



